In [37]:
import json
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point
pd.options.display.max_columns = 1000 # to see all columns
import sddk

In [3]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [41]:
EDHCS = sddk.read_file("SDAM_data/EDHCS/EDHCS_v2-0.json", "df", conf)

In [42]:
EDHCSg = gpd.GeoDataFrame(EDHCS, geometry=gpd.points_from_xy(EDHCS["geometry"].apply(lambda x: x[0]), EDHCS["geometry"].apply(lambda x: x[1])), crs={"init": "epsg:3857"})

/home/kasev/social_diversity/socdiv_venv/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



In [43]:
len(EDHCS.columns)

110

In [44]:
EDHCSg_sample = EDHCSg[100000:101000]

In [49]:
def gdf_to_geojson(gdf_input):
    # serialize geometry:
    gdf = gdf_input.copy()
    gdf["geometry"] = gdf["geometry"].apply(lambda x: eval(json.dumps(shapely.geometry.mapping(x))))
    # gdf into dict object in geojson structure:
    for col in gdf.columns:
        if list in [type(ins) for ins in gdf[col]]:
            gdf[col] = gdf[col].apply(lambda x: str(x))
    dict_list_object = [{"type" : "Feature", "geometry" : el["geometry"], "properties": {key:val for key, val in el.items() if key != 'geometry'}} for el in gdf.to_dict("records")]
    geojson_structure = {"type": "FeatureCollection", "features": dict_list_object}
    return geojson_structure

In [50]:
with open("../data/large_data/EDHCS_sample.geojson", "w") as outfile: 
    json.dump(gdf_to_geojson(EDHCSg_sample), outfile)

/home/kasev/social_diversity/socdiv_venv/lib/python3.6/site-packages/geopandas/geodataframe.py:1321: UserWarning:

Geometry column does not contain geometry.



In [51]:
EDHCS_sample_back = gpd.read_file("../data/large_data/EDHCS_sample.geojson", driver='GeoJSON')

In [52]:
len(EDHCS_sample_back.columns)

110

In [53]:
EDHCS_sample_back.head(5)

,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,status_notation,inscr_process,notes_references,notes_comments,inscription,inscription_stripped_final,Links,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment,EDH-ID,language_EDCS,clean_text_interpretive_word_EDCS,responsible_individual,type_of_inscription,letter_size,literature,work_status,height,diplomatic_text,people,depth,material,type_of_monument,province_label,width,transcription,country,uri,findspot_ancient,last_update,modern_region,findspot_modern,edh_geography_uri,commentary,trismegistos_uri,external_image_uris,fotos,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military,not_after,language,not_before,coordinates,clean_text_interpretive_word,urban_context,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,geometry
0,EDCS-03700668,"AE 1993, 01268",Dalmatia,Dalmatia,Podgrade / Asseria,"['Podgrade', 'Asseria']",150,,"['mulieres', 'tituli sepulcrales', 'tria nomina']",tituli sepulcrales,"['mulieres', 'tria nomina']",{},,,Aufidiae / C(ai) f(iliae) Balbin(a)e / an(noru...,Aufidiae / C(ai) f(iliae) Balbin(a)e / an(noru...,http://db.edcs.eu/epigr/partner.php?s_language...,1.0,150,mulieres; tituli sepulcrales; tria nomina,44.020343,15.669073,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,None,None,HD040124,None,Aufidiae Cai filiae Balbinae annorum V Caesia ...,Feraudi,epitaph,3.5-2.5 cm,"AE 1993, 1268.; I. Fadić, Diadora 12, 1990, 26...",checked with photo,(81) cm,AVFIDIAE / C F BALBINE / AN V CAESIA / L F PAV...,"[{'person_id': '1', 'nomen': 'Aufidia', 'gende...",None,None,None,Dalmatia,52 cm,Aufidiae / C(ai) f(iliae) Balbin(a)e / an(noru...,Croatia,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Asseria,2009-09-28,Zadarska županija,Benkovac,https://edh-www.adw.uni-heidelberg.de/edh/geog...,Denkmalform: sog. Liburnischer Cippus.,https://www.trismegistos.org/text/184041,http://lupa.at/23260,{},184041,"['http://www.trismegistos.org/place/011624', '...",Aufidiae Cai filiae Balbinae a...,1 AD – 150 AD,21,unbestimmt,2.0,unbestimmt,138,unbestimmt,2000.0,92,Grabinschrift,epitaph,Certain,81.0,52.0,NaN,NULL,NULL,NULL,Dalmatia,Certain,Croatia,Certain,Asseria,Certain,Zadarska županija,Certain,Benkovac,Certain,"Podgrađe, bei",Certain,1 AD – 150 AD,Aufidiae C f Balbine an V Caesia L f Paula avi...,Aufidiae Cai filiae Balbinae annorum V Caesia ...,"Podgrađe, bei",1911,"Zadar, Arh. Muz.",None,None,None,None,150.0,Latin,1.0,"[15.668, 44.0103]",Aufidiae Cai filiae Balbinae annorum V Caesia ...,rest,False,Asseria,511,458.0,"[15.680565, 44.017634]",minor,0.014549,POINT (15.66800 44.01030)
1,EDCS-03700669,"AE 1993, 01269",Dalmatia,Dalmatia,Podgrade / Asseria,"['Podgrade', 'Asseria']",300,,tituli sepulcrales,tituli sepulcrales,{},{},,,[3] C(ai) f(ilio?) / [3]ul[3],[3] C(ai) f(ilio?) / [3]ul[3],http://db.edcs.eu/epigr/partner.php?s_language...,1.0,300,tituli sepulcrales,44.020343,15.669073,None,None,None,HD040129,None,Cai filio ul,Feraudi,epitaph,3 cm,"AE 1993, 1269.; I. Fadić, Diadora 12, 1990, 26...",checked with photo,(70) cm,] / C F / [ ]VL[ ],"[{'name': '[---] C.f. [---]ul[---]', 'cognomen...",None,None,None,Dalmatia,60 cm,------]

In [54]:
set([type(el) for el in EDHCSg_sample["place_list"]])

{list, str}

In [55]:
set([type(el) for el in EDHCS_sample_back["place_list"]])

{str}